In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

2022-03-24 16:30:25.130855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 16:30:25.162516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-24 16:30:25.162533: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-24 16:30:25.162916: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]


In [3]:
#k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

In [4]:
paths = ['/home/wander/OtherProjects/har_flower/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            'neurons_1' : 32, 
            'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    test_results, ba = har.run()
    #model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
print(max(bas))


Epoch 1/40
254/254 [==============================] - 1s 2ms/step - loss: 302665.0312 - avg_multilabel_BA_2: 0.7587 - val_loss: 0.3643 - val_avg_multilabel_BA_2: 0.7439
Epoch 2/40
254/254 [==============================] - 0s 907us/step - loss: 0.2811 - avg_multilabel_BA_2: 0.7510 - val_loss: 0.3946 - val_avg_multilabel_BA_2: 0.7455
Epoch 3/40
254/254 [==============================] - 0s 913us/step - loss: 0.2819 - avg_multilabel_BA_2: 0.7496 - val_loss: 0.3387 - val_avg_multilabel_BA_2: 0.7454
Epoch 4/40
254/254 [==============================] - 0s 903us/step - loss: 0.2821 - avg_multilabel_BA_2: 0.7498 - val_loss: 0.3362 - val_avg_multilabel_BA_2: 0.7459
Epoch 5/40
254/254 [==============================] - 0s 894us/step - loss: 0.2827 - avg_multilabel_BA_2: 0.7487 - val_loss: 0.3640 - val_avg_multilabel_BA_2: 0.7462
Epoch 6/40
254/254 [==============================] - 0s 922us/step - loss: 0.2827 - avg_multilabel_BA_2: 0.7482 - val_loss: 0.3498 - val_avg_multilabel_BA_2: 0.7464
E

Epoch 9/40
199/199 [==============================] - 0s 910us/step - loss: 0.2744 - avg_multilabel_BA_2: 0.7212 - val_loss: 0.2594 - val_avg_multilabel_BA_2: 0.7199
Epoch 10/40
199/199 [==============================] - 0s 911us/step - loss: 0.2745 - avg_multilabel_BA_2: 0.7187 - val_loss: 0.2652 - val_avg_multilabel_BA_2: 0.7174
Epoch 11/40
199/199 [==============================] - 0s 903us/step - loss: 0.2744 - avg_multilabel_BA_2: 0.7162 - val_loss: 0.2623 - val_avg_multilabel_BA_2: 0.7150
Epoch 12/40
199/199 [==============================] - 0s 907us/step - loss: 0.2738 - avg_multilabel_BA_2: 0.7138 - val_loss: 0.2779 - val_avg_multilabel_BA_2: 0.7126
Epoch 13/40
199/199 [==============================] - 0s 917us/step - loss: 0.2744 - avg_multilabel_BA_2: 0.7114 - val_loss: 0.2754 - val_avg_multilabel_BA_2: 0.7102
Epoch 14/40
199/199 [==============================] - 0s 920us/step - loss: 0.2739 - avg_multilabel_BA_2: 0.7090 - val_loss: 0.2628 - val_avg_multilabel_BA_2: 0.7079

Epoch 17/40
199/199 [==============================] - 0s 925us/step - loss: 0.2752 - avg_multilabel_BA_2: 0.6413 - val_loss: 0.2883 - val_avg_multilabel_BA_2: 0.6408
Epoch 18/40
199/199 [==============================] - 0s 917us/step - loss: 0.2746 - avg_multilabel_BA_2: 0.6403 - val_loss: 0.2625 - val_avg_multilabel_BA_2: 0.6397
Epoch 19/40
199/199 [==============================] - 0s 918us/step - loss: 0.2742 - avg_multilabel_BA_2: 0.6392 - val_loss: 0.2719 - val_avg_multilabel_BA_2: 0.6387
Epoch 20/40
199/199 [==============================] - 0s 911us/step - loss: 0.2756 - avg_multilabel_BA_2: 0.6382 - val_loss: 0.2668 - val_avg_multilabel_BA_2: 0.6377
Epoch 21/40
199/199 [==============================] - 0s 915us/step - loss: 0.2747 - avg_multilabel_BA_2: 0.6372 - val_loss: 0.2720 - val_avg_multilabel_BA_2: 0.6367
Epoch 22/40
199/199 [==============================] - 0s 921us/step - loss: 0.2750 - avg_multilabel_BA_2: 0.6362 - val_loss: 0.2804 - val_avg_multilabel_BA_2: 0.635

Epoch 25/40
199/199 [==============================] - 0s 913us/step - loss: 0.2728 - avg_multilabel_BA_2: 0.6038 - val_loss: 0.2919 - val_avg_multilabel_BA_2: 0.6035
Epoch 26/40
199/199 [==============================] - 0s 913us/step - loss: 0.2731 - avg_multilabel_BA_2: 0.6032 - val_loss: 0.2911 - val_avg_multilabel_BA_2: 0.6029
Epoch 27/40
199/199 [==============================] - 0s 910us/step - loss: 0.2721 - avg_multilabel_BA_2: 0.6026 - val_loss: 0.2820 - val_avg_multilabel_BA_2: 0.6024
Epoch 28/40
199/199 [==============================] - 0s 908us/step - loss: 0.2735 - avg_multilabel_BA_2: 0.6021 - val_loss: 0.2722 - val_avg_multilabel_BA_2: 0.6018
Epoch 29/40
199/199 [==============================] - 0s 932us/step - loss: 0.2734 - avg_multilabel_BA_2: 0.6015 - val_loss: 0.2654 - val_avg_multilabel_BA_2: 0.6013
Epoch 30/40
199/199 [==============================] - 0s 926us/step - loss: 0.2730 - avg_multilabel_BA_2: 0.6010 - val_loss: 0.2834 - val_avg_multilabel_BA_2: 0.600

In [5]:
'''def nan_mse(y_actual, y_predicted):
    stack = tf.stack((tf.math.is_nan(y_actual), 
                      tf.math.is_nan(y_predicted)),
                     axis=1)
    is_nans = tf.keras.backend.any(stack, axis=1)
    per_instance = tf.where(is_nans,
                            tf.zeros_like(y_actual),
                            tf.square(tf.subtract(y_predicted, y_actual)))
    print(stack)
    print(per_instance)
    return tf.reduce_mean(per_instance, axis=0)

print(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'''

'def nan_mse(y_actual, y_predicted):\n    stack = tf.stack((tf.math.is_nan(y_actual), \n                      tf.math.is_nan(y_predicted)),\n                     axis=1)\n    is_nans = tf.keras.backend.any(stack, axis=1)\n    per_instance = tf.where(is_nans,\n                            tf.zeros_like(y_actual),\n                            tf.square(tf.subtract(y_predicted, y_actual)))\n    print(stack)\n    print(per_instance)\n    return tf.reduce_mean(per_instance, axis=0)\n\nprint(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'

In [6]:
har.data.x

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:temperature_ambient,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1445366534,1.023488,0.024622,-0.028726,0.066325,1.018790,1.021975,1.026449,0.765474,6.684317,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
1,1445366594,1.025689,0.065634,0.042226,0.129952,1.011969,1.020782,1.032243,1.446182,6.682553,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
2,1445366674,1.022835,0.039641,-0.052978,0.100098,1.020100,1.022686,1.025343,0.616602,6.683829,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3,1445366703,1.022907,0.008549,-0.015235,0.029863,1.020766,1.022855,1.025048,0.807471,6.684576,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
4,1445366761,1.023375,0.011622,0.017318,0.030025,1.020367,1.022504,1.024599,0.682306,6.684548,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,1445969636,1.012473,0.052780,0.064610,0.121541,1.003088,1.010756,1.019279,1.159897,6.683282,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3104,1445969661,1.013537,0.018170,-0.010494,0.032543,1.005406,1.013759,1.022427,1.792811,6.684451,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3105,1445969756,1.012776,0.008135,-0.006644,0.013948,1.008513,1.012893,1.017029,1.925567,6.684579,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3106,1445969782,1.013261,0.013788,0.008416,0.021793,1.006162,1.013186,1.019275,2.039158,6.684519,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2


In [7]:
har.data.x

,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,...,lf_measurements:temperature_ambient,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1445366534,1.023488,0.024622,-0.028726,0.066325,1.018790,1.021975,1.026449,0.765474,6.684317,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
1,1445366594,1.025689,0.065634,0.042226,0.129952,1.011969,1.020782,1.032243,1.446182,6.682553,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
2,1445366674,1.022835,0.039641,-0.052978,0.100098,1.020100,1.022686,1.025343,0.616602,6.683829,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3,1445366703,1.022907,0.008549,-0.015235,0.029863,1.020766,1.022855,1.025048,0.807471,6.684576,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
4,1445366761,1.023375,0.011622,0.017318,0.030025,1.020367,1.022504,1.024599,0.682306,6.684548,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,1445969636,1.012473,0.052780,0.064610,0.121541,1.003088,1.010756,1.019279,1.159897,6.683282,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3104,1445969661,1.013537,0.018170,-0.010494,0.032543,1.005406,1.013759,1.022427,1.792811,6.684451,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3105,1445969756,1.012776,0.008135,-0.006644,0.013948,1.008513,1.012893,1.017029,1.925567,6.684579,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
3106,1445969782,1.013261,0.013788,0.008416,0.021793,1.006162,1.013186,1.019275,2.039158,6.684519,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2
